In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

from scipy.optimize import minimize 

In [5]:
# problem 2
df = pd.read_csv('normal_samples.csv', index_col=0)
n = len(df)
mu = np.mean(df)
sigma_mle = np.std(df, ddof=0)
sigma = np.std(df, ddof=1)
# alternatively sigma = np.var(df, ddof=1)

t_a_n = st.t.ppf( 0.95, df=(n-1))

lower_ci = mu - t_a_n * sigma / np.sqrt(n)
upper_ci = mu + t_a_n * sigma / np.sqrt(n)
print(mu, sigma_mle, lower_ci, upper_ci)

values    0.301835
dtype: float64 values    0.258794
dtype: float64 values    0.180013
dtype: float64 values    0.423657
dtype: float64


In [8]:
# problem 3
X = pd.read_csv('accidents.csv', index_col=0)
X = X['values']

def log_likelihood(x, p, l1, l2):
    L = p * st.poisson.pmf(x, l1) + (1 - p) * st.poisson.pmf(x, l2)
    l = np.log(L)
    return np.sum(l)

def find_mle(X):
    l = lambda x: (-1) * log_likelihood(X, x[0], x[1], x[2])    
    x0 = [0.5, 1, 2]
    bds = ((0, 1.0), (0, None), (0, None))
    res = minimize(l, x0, bounds=bds)
    return res.x

def boostrap_ci(X, m, alpha):
    n = len(X)
    Y = np.zeros((m,3))
    x0 = find_mle(X)
    for i in range(0,m):
        X_resample = np.random.choice(X, replace=True, size=(n,))
        Y[i,:] = find_mle(X_resample)
    return (x0, np.percentile(Y, alpha/2.0, axis=0), np.percentile(Y, 100 - alpha/2, axis=0))

boostrap_ci(X, 200, 5)

(array([0.3422543 , 1.33184218, 3.07485365]),
 array([0.26363661, 1.15317247, 2.95435125]),
 array([0.42244286, 1.48735074, 3.20951916]))